# FESOM2 Data Analysis with Nereus

This notebook demonstrates the capabilities of the `nereus` package for working with FESOM2 model data.

We will cover:
1. Loading FESOM mesh and data
2. Exploring mesh properties
3. Plotting ocean temperature data
4. Sea ice diagnostics (area, volume, extent)
5. Ocean diagnostics (heat content, volume-weighted mean)
6. Hovmoller diagrams

In [ ]:
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
import nereus as nr

%matplotlib inline

## 1. Loading FESOM Mesh and Data

Nereus provides a convenient way to load FESOM2 meshes and data files. The mesh contains node coordinates, areas, and depth information that are essential for diagnostics.

In [ ]:
# Define paths
mesh_path = '/Users/nkolduno/PYTHON/DATA/CORE27_mesh'
data_path = '/Users/nkolduno/PYTHON/DATA/CORE27_data'

# Load mesh using the nereus FESOM submodule
mesh = nr.fesom.load_mesh(mesh_path)
print(mesh)

In [ ]:
# Load data files
data_temp = xr.open_dataset(f'{data_path}/temp.fesom.1958.nc')
data_conc = xr.open_dataset(f'{data_path}/a_ice.fesom.1958.nc')
data_thick = xr.open_dataset(f'{data_path}/m_ice.fesom.1958.nc')

print("Temperature dataset:")
print(data_temp)

## 2. Exploring Mesh Properties

The FesomMesh object provides access to all mesh information including coordinates, areas, and vertical structure.

In [ ]:
# Mesh dimensions
print(f"Number of 2D nodes: {mesh.n2d:,}")
print(f"Number of vertical levels: {mesh.nlev}")
print(f"Total 3D nodes: {mesh.n3d:,}")
print(f"Number of triangular elements: {len(mesh.elem):,}")

# Coordinate ranges
print(f"\nLongitude range: [{mesh.lon.min():.2f}, {mesh.lon.max():.2f}]")
print(f"Latitude range: [{mesh.lat.min():.2f}, {mesh.lat.max():.2f}]")

# Depth levels
print(f"\nDepth levels (first 10): {mesh.depth[:10]}")
print(f"Maximum depth: {mesh.depth.max():.1f} m")

In [ ]:
# Cell areas
total_area = mesh.area.sum()
earth_area = 4 * np.pi * 6371e3**2
print(f"Total mesh area: {total_area:.2e} m^2")
print(f"Earth surface area: {earth_area:.2e} m^2")
print(f"Mesh coverage: {100 * total_area / earth_area:.1f}%")

# Area statistics
print(f"\nCell area statistics:")
print(f"  Min: {mesh.area.min():.2e} m^2")
print(f"  Max: {mesh.area.max():.2e} m^2")
print(f"  Mean: {mesh.area.mean():.2e} m^2")

## 3. Plotting Ocean Temperature

Nereus provides powerful plotting capabilities for unstructured data, automatically handling regridding to regular grids for visualization.

In [ ]:
# Plot sea surface temperature (first time step, first level)
sst = data_temp['temp'].isel(time=0, nz1=0)

fig, ax, interp = nr.plot(
    sst.values, 
    mesh.lon, 
    mesh.lat,
    projection='rob',  # Robinson projection
    cmap='RdYlBu_r',
    vmin=-2,
    vmax=30,
    title='Sea Surface Temperature (January 1958)',
    colorbar_label='Temperature (°C)'
)
plt.tight_layout()
plt.show()

In [ ]:
# Plot temperature in Arctic region (North Polar Stereographic)
fig, ax, _ = nr.plot(
    sst.values, 
    mesh.lon, 
    mesh.lat,
    projection='npstere',  # North polar stereographic
    interpolator=interp,   # Reuse interpolator for efficiency
    cmap='RdYlBu_r',
    vmin=-2,
    vmax=15,
    title='Arctic Sea Surface Temperature',
    colorbar_label='Temperature (°C)'
)
plt.tight_layout()
plt.show()

In [ ]:
# Plot temperature at depth (~1000m level)
# Find the level closest to 1000m
level_1000m = np.argmin(np.abs(mesh.depth - 1000))
print(f"Plotting level {level_1000m} at depth {mesh.depth[level_1000m]:.1f} m")

temp_1000m = data_temp['temp'].isel(time=0, nz1=level_1000m)

fig, ax, _ = nr.plot(
    temp_1000m.values, 
    mesh.lon, 
    mesh.lat,
    projection='rob',
    cmap='RdYlBu_r',
    vmin=0,
    vmax=15,
    title=f'Temperature at {mesh.depth[level_1000m]:.0f}m depth',
    colorbar_label='Temperature (°C)'
)
plt.tight_layout()
plt.show()

## 4. Sea Ice Diagnostics

Nereus provides functions for computing sea ice metrics including ice area, ice volume, and ice extent.

In [ ]:
# Plot sea ice concentration (first time step)
sic = data_conc['a_ice'].isel(time=0)

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Arctic
fig1, ax1, _ = nr.plot(
    sic.values, 
    mesh.lon, 
    mesh.lat,
    projection='npstere',
    cmap='Blues',
    vmin=0,
    vmax=1,
    title='Arctic Sea Ice Concentration',
    colorbar_label='Concentration',
    ax=axes[0]
)

# Antarctic
fig2, ax2, _ = nr.plot(
    sic.values, 
    mesh.lon, 
    mesh.lat,
    projection='spstere',
    cmap='Blues',
    vmin=0,
    vmax=1,
    title='Antarctic Sea Ice Concentration',
    colorbar_label='Concentration',
    ax=axes[1]
)

plt.tight_layout()
plt.show()

In [ ]:
# Compute ice diagnostics for first time step
sic_t0 = data_conc['a_ice'].isel(time=0).values
sit_t0 = data_thick['m_ice'].isel(time=0).values

# Hemisphere masks
nh_mask = mesh.lat > 0
sh_mask = mesh.lat < 0

# Ice area (concentration-weighted)
nh_ice_area = nr.ice_area(sic_t0, mesh.area, mask=nh_mask)
sh_ice_area = nr.ice_area(sic_t0, mesh.area, mask=sh_mask)

# Ice extent (area with concentration > 15%)
nh_ice_extent = nr.ice_extent(sic_t0, mesh.area, threshold=0.15, mask=nh_mask)
sh_ice_extent = nr.ice_extent(sic_t0, mesh.area, threshold=0.15, mask=sh_mask)

# Ice volume
nh_ice_vol = nr.ice_volume(sit_t0, mesh.area, concentration=sic_t0, mask=nh_mask)
sh_ice_vol = nr.ice_volume(sit_t0, mesh.area, concentration=sic_t0, mask=sh_mask)

# Convert to common units
km2_to_m2 = 1e6
km3_to_m3 = 1e9

print("Sea Ice Diagnostics (January 1958)")
print("=" * 45)
print(f"{'Metric':<25} {'NH':>10} {'SH':>10}")
print("-" * 45)
print(f"{'Ice Area (10^6 km^2)':<25} {nh_ice_area/km2_to_m2/1e6:>10.2f} {sh_ice_area/km2_to_m2/1e6:>10.2f}")
print(f"{'Ice Extent (10^6 km^2)':<25} {nh_ice_extent/km2_to_m2/1e6:>10.2f} {sh_ice_extent/km2_to_m2/1e6:>10.2f}")
print(f"{'Ice Volume (10^3 km^3)':<25} {nh_ice_vol/km3_to_m3/1e3:>10.2f} {sh_ice_vol/km3_to_m3/1e3:>10.2f}")

In [ ]:
# Compute time series of ice area and extent for both hemispheres
n_times = len(data_conc.time)
time = data_conc.time.values

nh_area_ts = np.zeros(n_times)
sh_area_ts = np.zeros(n_times)
nh_extent_ts = np.zeros(n_times)
sh_extent_ts = np.zeros(n_times)

for t in range(n_times):
    sic_t = data_conc['a_ice'].isel(time=t).values
    nh_area_ts[t] = nr.ice_area(sic_t, mesh.area, mask=nh_mask)
    sh_area_ts[t] = nr.ice_area(sic_t, mesh.area, mask=sh_mask)
    nh_extent_ts[t] = nr.ice_extent(sic_t, mesh.area, threshold=0.15, mask=nh_mask)
    sh_extent_ts[t] = nr.ice_extent(sic_t, mesh.area, threshold=0.15, mask=sh_mask)

# Plot time series
fig, axes = plt.subplots(2, 1, figsize=(12, 8), sharex=True)

# Ice area
axes[0].plot(time, nh_area_ts/km2_to_m2/1e6, 'b-', label='Northern Hemisphere', linewidth=2)
axes[0].plot(time, sh_area_ts/km2_to_m2/1e6, 'r-', label='Southern Hemisphere', linewidth=2)
axes[0].set_ylabel('Ice Area (10$^6$ km$^2$)')
axes[0].legend()
axes[0].set_title('Sea Ice Area (1958)')
axes[0].grid(alpha=0.3)

# Ice extent
axes[1].plot(time, nh_extent_ts/km2_to_m2/1e6, 'b-', label='Northern Hemisphere', linewidth=2)
axes[1].plot(time, sh_extent_ts/km2_to_m2/1e6, 'r-', label='Southern Hemisphere', linewidth=2)
axes[1].set_ylabel('Ice Extent (10$^6$ km$^2$)')
axes[1].set_xlabel('Time')
axes[1].legend()
axes[1].set_title('Sea Ice Extent (>15% concentration)')
axes[1].grid(alpha=0.3)

plt.tight_layout()
plt.show()

## 5. Ocean Diagnostics

Nereus provides functions for computing ocean heat content and volume-weighted means.

In [ ]:
# Get temperature for first time step
temp_3d = data_temp['temp'].isel(time=0).values  # shape: (nz, nod2)

# Layer thicknesses
layer_thick = mesh.layer_thickness
print(f"Temperature shape: {temp_3d.shape}")
print(f"Layer thickness shape: {layer_thick.shape}")
print(f"Depth levels: {mesh.nlev}")

In [ ]:
# Compute global ocean heat content in different depth ranges
ohc_0_700 = nr.heat_content(
    temp_3d, mesh.area, layer_thick, mesh.depth,
    depth_max=700
)

ohc_0_2000 = nr.heat_content(
    temp_3d, mesh.area, layer_thick, mesh.depth,
    depth_max=2000
)

ohc_total = nr.heat_content(
    temp_3d, mesh.area, layer_thick
)

# Convert to ZJ (10^21 J)
zj = 1e21

print("Global Ocean Heat Content (January 1958)")
print("=" * 40)
print(f"OHC 0-700m:   {ohc_0_700/zj:>10.1f} ZJ")
print(f"OHC 0-2000m:  {ohc_0_2000/zj:>10.1f} ZJ")
print(f"OHC total:    {ohc_total/zj:>10.1f} ZJ")

In [ ]:
# Compute volume-weighted mean temperature in different depth ranges
mean_temp_upper = nr.volume_mean(
    temp_3d, mesh.area, layer_thick, mesh.depth,
    depth_max=500
)

mean_temp_deep = nr.volume_mean(
    temp_3d, mesh.area, layer_thick, mesh.depth,
    depth_min=1000, depth_max=3000
)

mean_temp_all = nr.volume_mean(
    temp_3d, mesh.area, layer_thick
)

print("Volume-Weighted Mean Temperature (January 1958)")
print("=" * 50)
print(f"Upper ocean (0-500m):      {mean_temp_upper:>8.2f} °C")
print(f"Deep ocean (1000-3000m):   {mean_temp_deep:>8.2f} °C")
print(f"Full depth:                {mean_temp_all:>8.2f} °C")

In [ ]:
# Compute OHC by latitude band
lat_bands = [
    ('Tropics (-23°S to 23°N)', (mesh.lat > -23) & (mesh.lat < 23)),
    ('Northern subtropics (23°N to 45°N)', (mesh.lat >= 23) & (mesh.lat < 45)),
    ('Southern subtropics (45°S to 23°S)', (mesh.lat > -45) & (mesh.lat <= -23)),
    ('Northern high latitudes (>45°N)', mesh.lat >= 45),
    ('Southern high latitudes (<45°S)', mesh.lat <= -45),
]

print("Ocean Heat Content by Latitude Band (0-2000m)")
print("=" * 55)
for name, mask in lat_bands:
    ohc = nr.heat_content(
        temp_3d, mesh.area, layer_thick, mesh.depth,
        depth_max=2000, mask=mask
    )
    print(f"{name:<35} {ohc/zj:>10.1f} ZJ")

## 6. Hovmoller Diagrams

Hovmoller diagrams show the evolution of a variable over time and either depth or latitude.

In [ ]:
# Time-depth Hovmoller for global mean temperature
temp_all = data_temp['temp'].values  # (time, nz, nod2)
time = np.arange(len(data_temp.time))  # Use indices for simplicity

# Compute Hovmoller data
t_out, z_out, hov_temp = nr.hovmoller(
    temp_all, mesh.area, time, mesh.depth, mode='depth'
)

# Plot
fig, ax = nr.plot_hovmoller(
    t_out, z_out, hov_temp,
    mode='depth',
    cmap='RdYlBu_r',
    title='Global Mean Temperature Profile (1958)',
    colorbar_label='Temperature (°C)',
    figsize=(12, 6)
)
ax.set_xlabel('Month')
ax.set_ylim([2000, 0])  # Focus on upper 2000m
plt.tight_layout()
plt.show()

In [ ]:
# Time-latitude Hovmoller for SST
sst_all = data_temp['temp'].isel(nz1=0).values  # (time, nod2)

t_out, lat_out, hov_sst = nr.hovmoller(
    sst_all, mesh.area, time, lat=mesh.lat, mode='latitude',
    lat_bins=np.arange(-90, 95, 5)
)

fig, ax = nr.plot_hovmoller(
    t_out, lat_out, hov_sst,
    mode='latitude',
    cmap='RdYlBu_r',
    vmin=-2,
    vmax=30,
    title='Zonal Mean SST Evolution (1958)',
    colorbar_label='Temperature (°C)',
    figsize=(12, 6)
)
ax.set_xlabel('Month')
plt.tight_layout()
plt.show()

In [ ]:
# Time-latitude Hovmoller for sea ice concentration
sic_all = data_conc['a_ice'].values  # (time, nod2)

t_out, lat_out, hov_sic = nr.hovmoller(
    sic_all, mesh.area, time, lat=mesh.lat, mode='latitude',
    lat_bins=np.arange(-90, 95, 5)
)

fig, ax = nr.plot_hovmoller(
    t_out, lat_out, hov_sic,
    mode='latitude',
    cmap='Blues',
    vmin=0,
    vmax=1,
    title='Zonal Mean Sea Ice Concentration (1958)',
    colorbar_label='Concentration',
    figsize=(12, 6)
)
ax.set_xlabel('Month')
plt.tight_layout()
plt.show()

## Summary

This notebook demonstrated the key capabilities of nereus for FESOM2 data analysis:

1. **Mesh Loading**: `nr.fesom.load_mesh()` loads FESOM mesh with coordinates, areas, and depth information

2. **Plotting**: `nr.plot()` provides easy visualization with automatic regridding and various projections

3. **Sea Ice Diagnostics**:
   - `nr.ice_area()` - Total sea ice area (concentration-weighted)
   - `nr.ice_extent()` - Sea ice extent (area above threshold)
   - `nr.ice_volume()` - Total sea ice volume

4. **Ocean Diagnostics**:
   - `nr.heat_content()` - Ocean heat content calculation
   - `nr.volume_mean()` - Volume-weighted mean in depth ranges

5. **Hovmoller Diagrams**:
   - `nr.hovmoller()` - Compute time-depth or time-latitude data
   - `nr.plot_hovmoller()` - Visualization of Hovmoller diagrams

In [ ]:
# Close datasets
data_temp.close()
data_conc.close()
data_thick.close()